In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")

In [2]:
import pyskip
import pyskip_3d

In [32]:
%%time
t = pyskip.Tensor((100, 30, 30), 0)
t[:, 0, :] = 1
t[20:80, 1, 10:20] = 2
t[40:50, 2:20, 12:18] = 3

Wall time: 0 ns


In [23]:
config = pyskip_3d.VoxelConfig()
config[0] = pyskip_3d.EmptyVoxel()
config[1] = pyskip_3d.ColorVoxel(128, 66, 0)
config[2] = pyskip_3d.ColorVoxel(0, 50, 150)
config[3] = pyskip_3d.ColorVoxel(20, 80, 20)

In [33]:
%%time
mesh = pyskip_3d.generate_mesh(config, t._tensor)

Wall time: 5.03 ms


In [34]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [35]:
%%time
geometry = BufferGeometry(
    attributes={
        "position": BufferAttribute(
            np.array(mesh.positions, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "index": BufferAttribute(
            np.array(mesh.triangles, dtype='uint16'),
            normalized=False,
        ),
        "color": BufferAttribute(
            np.array(mesh.colors, dtype='float32').reshape(-1, 3) / 255.0,
        ),
    },
)

Wall time: 23 ms


In [36]:
%%time
geometry.exec_three_obj_method("computeVertexNormals")

Wall time: 0 ns


In [37]:
%%time
material = ShaderMaterial(
    uniforms={
        "Ka": { "value": [0.3, 0.3, 0.3] },
        "Kd": { "value": [1.0, 1.0, 1.0] },
        "Ks": { "value": [0.8, 0.8, 0.8] },
        "LightPosition": { "value": [10.0, 0.0, 0.0, 1.0] },
        "Shininess": { "value": 200.0 },
    },
    vertexShader="""
        varying vec3 _normal;
        varying vec3 _position;
        varying vec3 _color;

        void main() {
            _normal = normalize(normalMatrix * normal);
            _position = vec3(modelViewMatrix * vec4(position, 1.0));
            _color = color;
            gl_Position = projectionMatrix * modelViewMatrix * vec4(position, 1.0);
        }
    """,
    fragmentShader="""
        varying vec3 _normal;
        varying vec3 _position;
        varying vec3 _color;

        uniform vec3 Ka;
        uniform vec3 Kd;
        uniform vec3 Ks;
        uniform vec4 LightPosition;
        uniform vec3 LightIntensity;
        uniform float Shininess;

        vec3 phong() {
            vec3 n = normalize(_normal);
            vec3 s = normalize(vec3(LightPosition) - _position);
            vec3 v = normalize(vec3(-_position));
            vec3 r = reflect(-s, n);

            vec3 ambient = Ka;
            vec3 diffuse = Kd * max(dot(s, n), 0.0);
            vec3 specular = Ks * pow(max(dot(r, v), 0.0), Shininess);
            
            vec3 base_color = _color * (ambient + diffuse);
            return base_color + specular;
        }

        void main() {
            gl_FragColor = vec4(phong(), 1.0);
        }
    """,
    vertexColors="VertexColors",
)

Wall time: 3 ms


In [38]:
%%time
js_mesh = Mesh(
    geometry=geometry,
    material=material,
    position=[0, 0, 0],
)

Wall time: 3 ms


In [39]:
%%time
camera = PerspectiveCamera(position=[3.0, 3.0, 3.0], fov=20.0)

scene = Scene(children=[js_mesh, camera])

Wall time: 5 ms


In [40]:
renderer = Renderer(
    camera=camera, 
    background="black", 
    background_opacity=1,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=984,
    height=984,
)

display(renderer)

Renderer(camera=PerspectiveCamera(fov=20.0, position=(3.0, 3.0, 3.0), projectionMatrix=(5.67128181961771, 0.0,…

Wall time: 9 ms
